In [216]:
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [217]:
traindata = pd.read_csv('data/mitbih_train.csv',header=None)
testdata = pd.read_csv('data/mitbih_test.csv',header=None)

In [218]:
traindata

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87550,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87551,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87552,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [219]:
testdata

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.758264,0.111570,0.000000,0.080579,0.078512,0.066116,0.049587,0.047521,0.035124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.908425,0.783883,0.531136,0.362637,0.366300,0.344322,0.333333,0.307692,0.296703,0.300366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.730088,0.212389,0.000000,0.119469,0.101770,0.101770,0.110619,0.123894,0.115044,0.132743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000000,0.910417,0.681250,0.472917,0.229167,0.068750,0.000000,0.004167,0.014583,0.054167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.570470,0.399329,0.238255,0.147651,0.000000,0.003356,0.040268,0.080537,0.070470,0.090604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21887,0.928736,0.871264,0.804598,0.742529,0.650575,0.535632,0.394253,0.250575,0.140230,0.102299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21888,0.802691,0.692078,0.587444,0.446936,0.318386,0.189836,0.118087,0.077728,0.112108,0.152466,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21889,1.000000,0.967359,0.620178,0.347181,0.139466,0.089021,0.103858,0.100890,0.106825,0.100890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21890,0.984127,0.567460,0.607143,0.583333,0.607143,0.575397,0.575397,0.488095,0.392857,0.238095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [220]:
scaler = StandardScaler()

In [221]:
y_test=testdata[len(testdata.columns)-1]
testdata = testdata.drop(testdata.columns[[len(testdata.columns)-1]], axis=1) #Remove the last column
x_test = scaler.fit_transform(testdata)

In [222]:
x_test

array([[ 0.45017238, -0.01663683, -1.37839723, ..., -0.08819499,
        -0.08446082, -0.0835278 ],
       [ 0.05975141,  0.10052673,  0.45723507, ..., -0.08819499,
        -0.08446082, -0.0835278 ],
       [-0.70056768, -2.51315818, -1.86652611, ..., -0.08819499,
        -0.08446082, -0.0835278 ],
       ...,
       [ 0.45017238,  0.93964253,  0.84680317, ..., -0.08819499,
        -0.08446082, -0.0835278 ],
       [ 0.38249941, -0.88926633,  0.78977321, ..., -0.08819499,
        -0.08446082, -0.0835278 ],
       [ 0.33919461,  0.69209661,  1.92014929, ..., -0.08819499,
        -0.08446082, -0.0835278 ]])

In [223]:
y_test

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
21887    4.0
21888    4.0
21889    4.0
21890    4.0
21891    4.0
Name: 187, Length: 21892, dtype: float64

In [224]:
y_train=traindata[len(traindata.columns)-1]
traindata = traindata.drop(traindata.columns[[len(testdata.columns)-1]], axis=1) #Remove the last column
x_train = scaler.fit_transform(traindata)

In [225]:
x_train

array([[ 0.36354765,  0.75879683,  1.13240614, ..., -0.09259618,
        -0.08959366, -0.41408817],
       [ 0.28954748,  0.47376846,  0.16526879, ..., -0.09259618,
        -0.08959366, -0.41408817],
       [ 0.45511319, -0.44497542, -1.04479413, ..., -0.09259618,
        -0.08959366, -0.41408817],
       ...,
       [ 0.06543057, -0.60263006,  0.75645932, ..., -0.09259618,
        -0.08959366,  3.08492986],
       [-0.13337832, -0.50759535,  1.85477435, ..., -0.09259618,
        -0.08959366,  3.08492986],
       [ 0.04626936,  0.39549766,  1.65735418, ..., -0.09259618,
        -0.08959366,  3.08492986]])

In [226]:
y_train

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
87549    4.0
87550    4.0
87551    4.0
87552    4.0
87553    4.0
Name: 187, Length: 87554, dtype: float64

In [227]:
class ModelResults:
        def __init__(self,modelName=None, accuracy=None, recall=None, precision=None):
            self.modelName = modelName
            self.accuracy = accuracy
            self.recall = recall
            self.precision = precision
        def __toString__(self):
            return f"ModelResults(model name={self.modelName}, accuracy={self.accuracy}, recall={self.recall}, precision={self.precision})"

In [228]:
def trainAndTestModel(modelName,model):
    print("#######################################")
    print(modelName)
    print("#######################################")
    print("\n")
    
    model_accuracy = 0
    model_recall = np.array([0., 0., 0.,0.,0.])
    model_precision = np.array([0., 0., 0.,0.,0.])
    
    try:
        #Train 
        model.fit(x_train, y_train)
        #Test
        y_pred = model.predict(x_test)
        
        # Model performance    
        #print(classification_report(y_test, y_pred))
        
        # Performance scores
        model_accuracy += accuracy_score(y_test, y_pred)
        model_recall += recall_score(y_test, y_pred, average = None)
        model_precision += precision_score(y_test, y_pred, average = None)
        print(f"{modelName}, Done Training and testModel")
        print("\n")
        return modelName,model_accuracy, model_recall, model_precision
    
    except Exception as e:
        print(f"An error occurred during the train-test: {e}")
        return modelName,model_accuracy, model_recall, model_precision

    

In [229]:
models = {
    "SVC": SVC(kernel='linear', C=1.0),  # C is the regularization parameter
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, max_depth=5),  # n_estimators is the number of trees, max_depth is the max depth of the tree
    "GradientBoostingClassifier": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1),  # learning_rate shrinks the contribution of each tree
    "LogisticRegression": LogisticRegression(C=1.0, solver='liblinear'),  # C is the inverse of regularization strength, solver is the algorithm to use in the optimization problem
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=4),  # max_depth is the maximum depth of the tree
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors=5),  # n_neighbors is the number of neighbors to use
    "GaussianNB": GaussianNB(var_smoothing=1e-9),  # var_smoothing is the portion of the largest variance of all features that is added to variances for calculation stability
    "AdaBoostClassifier": AdaBoostClassifier(n_estimators=50, learning_rate=1.0)  # n_estimators is the number of trees, learning_rate shrinks the contribution of each classifier
}

In [230]:
n_folds=5
acc = 0
recall = np.array([0., 0., 0.,0.,0.])
precision = np.array([0., 0., 0.,0.,0.])

models_results=[]
for name,model in models.items():
    model_name,model_accuracy, model_recall, model_precision=trainAndTestModel(modelName=name,model=model)
    models_results.append(ModelResults(model_name,model_accuracy, model_recall, model_precision))
    


#######################################
SVC
#######################################


#######################################
RandomForestClassifier
#######################################




C:\Python\PyProjects\ECG-Heartbeat-Categorization\ECG Classificator\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#######################################
LogisticRegression
#######################################


#######################################
DecisionTreeClassifier
#######################################


#######################################
GaussianNB
#######################################




C:\Python\PyProjects\ECG-Heartbeat-Categorization\ECG Classificator\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [231]:
for results in models_results:
    print(results.__toString__())

ModelResults(model name=SVC, accuracy=0.8238625982093916, recall=[0.99541892 0.00179856 0.         0.         0.        ], precision=[0.83095282 1.         0.         0.         0.        ])
ModelResults(model name=RandomForestClassifier, accuracy=0.9141695596564955, recall=[0.99977923 0.07194245 0.46063536 0.         0.74129353], precision=[0.9064705  0.95238095 0.99552239 0.         0.99582289])
ModelResults(model name=LogisticRegression, accuracy=0.8248218527315915, recall=[0.9947014  0.0557554  0.00276243 0.         0.        ], precision=[0.83215588 0.79487179 0.26666667 0.         0.        ])
ModelResults(model name=DecisionTreeClassifier, accuracy=0.8238625982093916, recall=[0.99541892 0.00179856 0.         0.         0.        ], precision=[0.83095282 1.         0.         0.         0.        ])
ModelResults(model name=GaussianNB, accuracy=0.8238625982093916, recall=[0.99541892 0.00179856 0.         0.         0.        ], precision=[0.83095282 1.         0.         0.       